# Regularization with SciKit-Learn

Previously we created a new polynomial feature set and then applied our standard linear regression on it, but we can be smarter about model choice and utilize regularization.

Regularization attempts to minimize the RSS (residual sum of squares) *and* a penalty factor. This penalty factor will penalize models that have coefficients that are too large. Some methods of regularization will actually cause non useful features to have a coefficient of zero, in which case the model does not consider the feature.

Let's explore two methods of regularization, Ridge Regression and Lasso. We'll combine these with the polynomial feature set (it wouldn't be as effective to perform regularization of a model on such a small original feature set of the original X).

**IMPORTS**

In [1]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**Data and setup**

In [2]:
# df = pd.read_csv("Advertising.csv")

# X = df.drop("sales", axis=1)
# y = df["sales"]

df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

**Polynomial Conversion**

In [3]:
# from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures

In [4]:
# polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [5]:
# poly_features = polynomial_converter.fit_transform(X)
poly_features = polynomial_converter.fit_transform(X)

**Train | Test Split**

In [6]:
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

----
----

## Scaling the Data

While our particular data set has all the values in the same order of magnitude ($1000s of dollars spent), typically that won't be the case on a dataset, and since the mathematics behind regularized models will sum coefficients together, its important to standardize the features. Review the theory videos for more info, as well as a discussion on why we only **fit** to the training data, and **transform** on both sets separately.

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# help(StandardScaler)

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(X_train)

StandardScaler()

In [12]:
X_train = scaler.transform(X_train)

In [13]:
X_test = scaler.transform(X_test)

### Ridge Reggression

In [14]:
from sklearn.linear_model import Ridge

In [15]:
ridge_model = Ridge(alpha=10)

In [16]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [17]:
test_predictions = ridge_model.predict(X_test)

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [19]:
MAE = mean_absolute_error(y_test, test_predictions)

In [20]:
MAE

0.5774404204714169

In [21]:
MSE = mean_squared_error(y_test, test_predictions)

In [22]:
MSE

0.8003783071528395

In [23]:
RMSE = np.sqrt(MSE)

In [24]:
RMSE

0.8946386461319674

In [25]:
# Training set performance
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train, train_predictions)
MAE



0.5288348183025326

### Choosing an alpha value with Cross-Validation


In [26]:
from sklearn.linear_model import RidgeCV

In [27]:
# help(RidgeCV)

In [28]:
# Choosing a scoring: https://scikit-learn.org/stable/modules/model_evaluation.html
# Negative RMSE so all metrics follow convention "Higher is better"

# See all options: sklearn.metrics.SCORERS.keys()


In [29]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0,10.0), scoring = "neg_mean_absolute_error")


In [30]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [31]:
ridge_cv_model.alpha_

0.1

In [32]:
test_prediction = ridge_cv_model.predict(X_test)

In [33]:
MAE = mean_absolute_error(y_test,test_prediction)
MSE = mean_squared_error(y_test,test_prediction)
RMSE = np.sqrt(MSE)

In [34]:
MAE

0.427377488435447

In [35]:
RMSE

0.6180719926966545

In [36]:
# training set performance
train_cv_prediction = ridge_cv_model.predict(X_train)

In [37]:
train_MAE = mean_absolute_error(y_train, train_cv_prediction)
train_MAE

0.3094132105643277

In [38]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

### Lasso Regression

In [39]:
from sklearn.linear_model import LassoCV

In [40]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps = 0.1, n_alphas=100, cv=5)

In [41]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [42]:
lasso_cv_model.alpha_

0.49430709092258285

In [43]:
test_predictions = lasso_cv_model.predict(X_test)

In [44]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [45]:
MAE

0.6541723161252858

In [46]:
MSE

1.2787088713079866

In [47]:
RMSE

1.130800102276254

In [48]:
# Training Set Performance
# Training Set Performance
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.6912807140820701

In [49]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net

Elastic Net combines the penalties of ridge regression and lasso in an attempt to get the best of both worlds!

In [50]:
from sklearn.linear_model import ElasticNetCV

In [51]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [52]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], tol=0.01)

In [53]:
elastic_model.l1_ratio_

1.0

In [54]:
test_predictions = elastic_model.predict(X_test)

In [55]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [56]:
MAE

0.566326211756945

In [57]:
RMSE

0.7485546215633724

In [58]:
# Training Set Performance
# Training Set Performance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.4307582990472368

In [59]:
elastic_model.coef_

array([ 3.78993643,  0.89232919,  0.28765395, -1.01843566,  2.15516144,
       -0.3567547 , -0.271502  ,  0.09741081,  0.        , -1.05563151,
        0.2362506 ,  0.07980911,  1.26170778,  0.01464706,  0.00462336,
       -0.39986069,  0.        ,  0.        , -0.05343757])